<a href="https://colab.research.google.com/github/aknip/Langchain-etc./blob/main/Langchain-Agent-Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain Agent Insurance - using OpenAI functions

Different agents and code implementations how to run Langchain agents using OpenAI functions: Single prompt, chat with memory, from command line or in Gradio web app, using AgentExecutor or your own custom loop.

## How to use
1. Run all cells of group "Basic setup for all agents, tools and apps"
2. Run one of the two agent/tool groups
3. Run different code implementations after that: CLI, Gradio, custom loops...

For more information see https://levelup.gitconnected.com/building-a-powerful-agent-has-no-challenge-today-774e27be818d

# Basic setup for all agents, tools and apps

In [ ]:
import json
import os
from getpass import getpass
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

In [ ]:
!pip install langchain==0.0.311 langchain-experimental==0.0.27 langchainhub==0.1.13 google-search-results==2.4.2 wikipedia==1.4.0 openai==0.28.1 gradio==3.47.1 -q
%load_ext gradio



In [ ]:
from langchain.agents import Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper, SQLDatabase, WikipediaAPIWrapper
from langchain_experimental.sql import SQLDatabaseChain
from langchain.tools import WikipediaQueryRun
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool, StructuredTool
from langchain.schema.agent import AgentFinish
from langchain.schema import BaseMessage, AIMessage, HumanMessage
import os
import langchain
import openai
import textwrap
import gradio as gr
import random
import time

# Agent 2: Calculate insurance premium

In [41]:
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential']
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')

def calculate_insurance_premium(insured_sum: int, industry: str, country: str = "Germany") -> float:
    """Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer."""
    premium = insured_sum*0.25
    result = str(premium) + " (for " + country + ")"
    return result

calc_tool = StructuredTool.from_function(calculate_insurance_premium, return_direct=False)

def show_history() -> str:
    """Show the history of the chat or conversation to the user."""
    chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
    chat_history = chat_history_string.split("§§§ ")
    msg_history_string = ""
    for chat_string in chat_history:
      tmp = chat_string.split("(content='")
      msg_history_string = msg_history_string + tmp[0] + " - " + tmp[1][:-2] + "\n"
    return msg_history_string

show_history_tool = StructuredTool.from_function(show_history, return_direct=True)

tools = [calc_tool, show_history_tool]
tools_as_openai_functions = [format_tool_to_openai_function(t) for t in tools]
tools_dict = {
    "calculate_insurance_premium": calc_tool,
    "show_history": show_history_tool
    }

llm_with_tools = llm.bind(
    functions=tools_as_openai_functions
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant."),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()


In [ ]:
# @title
# show calc_tool as OpenAI function
print(json.dumps(format_tool_to_openai_function(calc_tool), sort_keys=False, indent=2))

{
  "name": "calculate_insurance_premium",
  "description": "calculate_insurance_premium(insured_sum: int, industry: str, country: str = 'Germany') -> float - Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer.",
  "parameters": {
    "title": "calculate_insurance_premiumSchemaSchema",
    "type": "object",
    "properties": {
      "insured_sum": {
        "title": "Insured Sum",
        "type": "integer"
      },
      "industry": {
        "title": "Industry",
        "type": "string"
      },
      "country": {
        "title": "Country",
        "default": "Germany",
        "type": "string"
      }
    },
    "required": [
      "insured_sum",
      "industry"
    ]
  }
}


In [ ]:
# @title
demoSchema = {
  "name": "calculate_insurance_premium",
  "description": "calculate_insurance_premium(insured_sum: int, industry: str, country: str = 'Germany') -> float - Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer.",
  "parameters": {
    "title": "calculate_insurance_premiumSchemaSchema",
    "type": "object",
    "properties": {
      "insured_sum": {
        "title": "Insured Sum",
        "type": "integer"
      },
      "industry": {
        "title": "Industry",
        "type": "string"
      },
      "country": {
        "title": "Country",
        "default": "Germany",
        "type": "string",
        "enum": ["positive", "negative"],
        "description": "The Country where the insured company is located"
      }
    },
    "required": [
      "insured_sum",
      "industry"
    ]
  }
}

# Chat loop

In [42]:
langchain.debug = False

intermediate_steps = []
chat_history = []

# manual loop / like AgentExecutor
def chat_manual_loop(prompt_text: str) -> str:
  while True:
      output = agent.invoke({
          "input": prompt_text,
          "intermediate_steps": intermediate_steps,
          "chat_history": chat_history
      })
      if isinstance(output, AgentFinish):
          final_result = output.return_values["output"]
          chat_history.append(HumanMessage(content=(prompt_text)))
          chat_history.append(AIMessage(content=(output.return_values["output"])))
          break
      else:
          print(output.tool, output.tool_input)
          tool = tools_dict[output.tool]
          observation = tool.run(output.tool_input)
          intermediate_steps.append((output, observation))
  return final_result

chat_response = chat_manual_loop("Kalkuliere die Versicherungsprämie für eine Versicherungssumme von 10000000")
print(chat_response)

Natürlich, ich kann das für Sie berechnen. Könnten Sie mir bitte noch mitteilen, in welcher Branche Sie tätig sind?


In [44]:
chat_response = chat_manual_loop("Die Branche ist Metallbau.")
print(chat_response)

Die Versicherungsprämie für eine Versicherungssumme von 10.000.000 € in der Branche Metallbau beträgt in Deutschland 2.500.000 €.


In [45]:
chat_response = chat_manual_loop("Zeige mir meine letzten Nachrichten.")
print(chat_response)

show_history {}
Hier sind Ihre letzten Nachrichten:

1. Sie: "Kalkuliere die Versicherungsprämie für eine Versicherungssumme von 10000000"
2. Ich: "Natürlich, ich kann das für Sie berechnen. Könnten Sie mir bitte noch mitteilen, in welcher Branche Sie tätig sind?"
3. Sie: "Die Branche ist Metallbau."
4. Ich: "Die Versicherungsprämie für eine Versicherungssumme von 10.000.000 € in der Branche Metallbau beträgt in Deutschland 2.500.000 €."


In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
messages = [{"role": "assistant", "content": "How can I help you?"}]
langchain.debug = False

In [ ]:
#prompt = "Give me the premium for an insured sum of 10000000"
#prompt = "The industry is chemical production"
prompt = "Show me my last messages"
messages.append({"role": "user", "content": prompt})
print("user: " + prompt)
response = agent_executor.invoke({"input": prompt})
msg = {"role": "assistant", "content": response["output"]}
messages.append(msg)
print(textwrap.fill("assistant: " + response["output"], 80))
print("\nHistory:")
for msg in messages:
    #st.chat_message(msg["role"]).write(msg["content"])
    print(textwrap.fill(msg["role"] + ": " + msg["content"], 80))

user: Show me my last messages


> Entering new AgentExecutor chain...

Invoking: `show_history` with `{}`


HumanMessage - Give me the premium for an insured sum of 10000000
AIMessage - Sure, I can help with that. However, I also need to know the industry of the customer to calculate the insurance premium. Could you please provide that information?
HumanMessage - The industry is chemical production
AIMessage - The premium for an insured sum of 10,000,000 in the chemical production industry in Germany is 2,500,000.
HumanMessage - Show me my last messages
AIMessage - HumanMessage - Give me the premium for an insured sum of 10000000\nAIMessage - Sure, I can help with that. However, I also need to know the industry of the customer to calculate the insurance premium. Could you please provide that information?\nHumanMessage - The industry is chemical production\nAIMessage - The premium for an insured sum of 10,000,000 in the chemical production industry in Germany is 2,500,000.\n



> Finis

In [ ]:
show_history()

'HumanMessage - Give me the premium for an insured sum of 10000000\nAIMessage - Sure, I can help with that. However, I also need to know the industry of the customer to calculate the insurance premium. Could you please provide that information?\nHumanMessage - The industry is chemical production\nAIMessage - The premium for an insurance with an insured sum of 10,000,000 in the chemical production industry in Germany is 2,500,000.\n'

In [ ]:
print("Memory from response object:")
chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
chat_history = chat_history_string.split("§§§ ")
msg_history = []
for chat_string in chat_history:
  tmp = chat_string.split("(content='")
  msg_history.append({"role": tmp[0], "content": tmp[1][:-2]})
#for msg in msg_history:
#  print(msg)
print('\n'.join(str(x) for x in msg_history))

Memory from response object:
{'role': 'HumanMessage', 'content': 'Give me the premium for an insured sum of 10000000'}
{'role': 'AIMessage', 'content': 'Sure, I can help with that. However, I also need to know the industry of the customer to calculate the insurance premium. Could you please provide that information?'}
{'role': 'HumanMessage', 'content': 'The industry is chemical production'}
{'role': 'AIMessage', 'content': 'The premium for an insured sum of 10,000,000 in the chemical production industry in Germany is 2,500,000.'}


# Gradio Chat App for Agent

In [ ]:
# Init agent and memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
messages = [{"role": "assistant", "content": "How can I help you?"}]

#
# Gradio app
#

# Theming
theme = gr.themes.Default(
    primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
)
# Styling: Change max width
css = """
  .gradio-container {max-width: 800px!important}
  .vspacer1 {margin-top: 20px}
"""

with gr.Blocks(theme=theme, css=css) as demo:

    gr.Markdown("# Agent Chat", elem_classes="vspacer1")

    with gr.Tab("Chat"):
      # https://www.gradio.app/docs/chatbot

      chatbot = gr.Chatbot(bubble_full_width=False)
      msg = gr.Textbox()
      clear = gr.ClearButton([msg, chatbot])

      def ask(message, chat_history):
          chat_history.append((message, None))
          messages.append({"role": "user", "content": message})
          return "", chat_history

      def respond(chat_history):
          prompt = chat_history[-1][0] # get prompt from history (last entry)
          response = agent_executor.invoke({"input": prompt})
          msg = {"role": "assistant", "content": response["output"]}
          messages.append(msg)
          chat_history.append((None, response["output"]))
          print("\n\nMemory from response object:")
          print(textwrap.fill(str(response["chat_history"]), 80))
          return chat_history

      msg.submit(ask, [msg, chatbot], [msg, chatbot], queue=False).then(
        respond, chatbot, chatbot
      )

demo.launch(quiet=True, share=False, debug=True)